In [315]:
import sys
import os
import re
import pprint

In [310]:
my_first_pat = '(\w+)@(\w+).edu'
#line17='email: pal at cs stanford edu'
#example [('ashishg', '@', 'stanford', '', '', '.', 'edu')]
#part0:(\w+\.?\w+?) == person name
#part1:(@|\bat\b|\(at\)|WHERE) == @
#part2:(\w+)
#part3:(\.|\;|\sdo?t\s|\sDOM\s)?
#part4:(\w+)?
#part5:(\.|\;|\sdo?t\s|\sDOM\s)?
#part6:(edu|com)\b
# and there may be \s* to connect each part
#we use . to connect part2,part4,and part6 to make it like stanford.edu
email_pat_0 = re.compile(r'\b(\w+\.?\w+?)\s*(@|\bat\b|\(at\)|WHERE|&#x40;)\s*(\w+)(\.|\;|\sdo?t\s|\sDOM\s)?\s*(\w+)?(\.|\;|\sdo?t\s|\sDOM\s)?\s*(edu|com)\b',re.I)
phone_pat = re.compile(r'\b(\d{3})(\-|\s+|\)\s*)(\d{3})(\-|\s+)(\d{4})\b')

#although email_pat is good enough,there are some exceptions
#to process these exceptions, we need to define email_pat_1

#line6='obfuscate(\'stanford.edu\',\'jurafsky\')</script>'
email_pat_1 = re.compile(r'obfuscate\(\'(\w+\.edu)\',\'(\w+)\'\)')



#line15=' <td class="value">ouster (followed by &ldquo;@cs.stanford.edu&rdquo;)</td>'
#line16='<td class="value">teresa.lynn (followed by "@stanford.edu")</td>'
#almost same with email_pat_1
#only add \(followed by\s*(&ldquo;)?
email_pat_2 = re.compile(r'\b(\w+\.?\w+?)\s*\(followed by\s*(&ldquo;)?\"?(@)\s*(\w+)(\.)?\s*(\w+)?(\.)?\s*(edu|com)\b',re.I)

#line22='d-l-w-h-@-s-t-a-n-f-o-r-d-.-e-d-u'
email_pat_3 = re.compile(r'([A-Za-z]-)+@-([A-Za-z]-)+\.-([A-Za-z]-?)+')

In [284]:
"""
TODO
This function takes in a filename along with the file object (actually
a StringIO object at submission time) and
scans its contents against regex patterns. It returns a list of
(filename, type, value) tuples where type is either an 'e' or a 'p'
for e-mail or phone, and value is the formatted phone number or e-mail.
The canonical formats are:
     (name, 'p', '###-###-#####')
     (name, 'e', 'someone@something')
If the numbers you submit are formatted differently they will not
match the gold answers

NOTE: ***don't change this interface***, as it will be called directly by
the submit script

NOTE: You shouldn't need to worry about this, but just so you know, the
'f' parameter below will be of type StringIO at submission time. So, make
sure you check the StringIO interface if you do anything really tricky,
though StringIO should support most everything.
"""
def process_file(name, f):
    # note that debug info should be printed to stderr
    # sys.stderr.write('[process_file]\tprocessing file: %s\n' % (path))
    res = []
    for line in f:
        email_matches_0 = re.findall(email_pat,line)
        phone_matches = re.findall(phone_pat,line)
        for m in email_matches_0:
            #[('ashishg', '@', 'stanford', '', '', '.', 'edu')]
            #join part2 part4 and part6 if they are not ''
            half_email = '.'.join(i for i in m[2::2]if(len(str(i)))!=0)
            #print half_email  
            email = '%s@%s' % (m[0],half_email.lower())
            #print(email)
            res.append((name,'e',email))
            
        for m in phone_matches:
            phone = '-'.join(m[::2])
            res.append((name,'p',phone))
            
        email_matches_1 = re.findall(email_pat_1,line)
        
        for m in email_matches_1:
            email = '%s@%s' % (m[1],m[0])
            res.append((name,'e',email))
            
        email_matches_2 = re.findall(email_pat_2,line)
        for m in email_matches_2:
            [('ouster', '&ldquo;', '@', 'cs', '.', 'stanford', '.', 'edu')]
            #join part2 part4 and part6 if they are not ''
            half_email = '.'.join(i for i in m[3::2]if(len(str(i)))!=0)
            #print half_email  
            email = '%s@%s' % (m[0],half_email.lower())
            #print(email)
            res.append((name,'e',email))
    return res


In [4]:
"""
You should not need to edit this function, nor should you alter
its interface as it will be called directly by the submit script
"""
def process_dir(data_path):
    # get candidates
    guess_list = []
    #The continue statement rejects all the remaining statements 
    #in the current iteration of the loop 
    #and moves the control back to the top of the loop.
    for fname in os.listdir(data_path):
        if fname[0] == '.':
            continue
        path = os.path.join(data_path,fname)
        f = open(path,'r')
        f_guesses = process_file(fname, f)
        guess_list.extend(f_guesses)
    return guess_list

In [5]:
"""
You should not need to edit this function.
Given a path to a tsv file of gold e-mails and phone numbers
this function returns a list of tuples of the canonical form:
(filename, type, value)
"""
def get_gold(gold_path):
    # get gold answers
    gold_list = []
    f_gold = open(gold_path,'r')
    for line in f_gold:
        gold_list.append(tuple(line.strip().split('\t')))
    return gold_list


In [6]:

"""
You should not need to edit this function.
Given a list of guessed contacts and gold contacts, this function
computes the intersection and set differences, to compute the true
positives, false positives and false negatives.  Importantly, it
converts all of the values to lower case before comparing
"""
def score(guess_list, gold_list):
    guess_list = [(fname, _type, value.lower()) for (fname, _type, value) in guess_list]
    gold_list = [(fname, _type, value.lower()) for (fname, _type, value) in gold_list]
    guess_set = set(guess_list)
    gold_set = set(gold_list)

    tp = guess_set.intersection(gold_set)
    fp = guess_set - gold_set
    fn = gold_set - guess_set

    pp = pprint.PrettyPrinter()
    #print 'Guesses (%d): ' % len(guess_set)
    #pp.pprint(guess_set)
    #print 'Gold (%d): ' % len(gold_set)
    #pp.pprint(gold_set)
    print 'True Positives (%d): ' % len(tp)
    pp.pprint(tp)
    print 'False Positives (%d): ' % len(fp)
    pp.pprint(fp)
    print 'False Negatives (%d): ' % len(fn)
    pp.pprint(fn)
    print 'Summary: tp=%d, fp=%d, fn=%d' % (len(tp),len(fp),len(fn))

In [17]:
"""
You should not need to edit this function.
It takes in the string path to the data directory and the
gold file
"""
def main(data_path, gold_path):
    guess_list = process_dir(data_path)
    gold_list =  get_gold(gold_path)
    score(guess_list, gold_list)

In [311]:
"""
commandline interface takes a directory name and gold file.
It then processes each file within that directory and extracts any
matching e-mails or phone numbers and compares them to the gold file
"""
if __name__ == '__main__':
    #if (len(sys.argv) == 1):
        main('../data/dev', '../data/devGOLD')
    #elif (len(sys.argv) == 3):
        #main(sys.argv[1],sys.argv[2])
    #else:
        #print 'usage:\tSpamLord.py <data_dir> <gold_file>'
        #sys.exit(0)

True Positives (116): 
set([('ashishg', 'e', 'ashishg@stanford.edu'),
     ('ashishg', 'e', 'rozm@stanford.edu'),
     ('ashishg', 'p', '650-723-1614'),
     ('ashishg', 'p', '650-723-4173'),
     ('ashishg', 'p', '650-814-1478'),
     ('balaji', 'e', 'balaji@stanford.edu'),
     ('bgirod', 'p', '650-723-4539'),
     ('bgirod', 'p', '650-724-3648'),
     ('bgirod', 'p', '650-724-6354'),
     ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
     ('cheriton', 'e', 'uma@cs.stanford.edu'),
     ('cheriton', 'p', '650-723-1131'),
     ('cheriton', 'p', '650-725-3726'),
     ('dabo', 'e', 'dabo@cs.stanford.edu'),
     ('dabo', 'p', '650-725-3897'),
     ('dabo', 'p', '650-725-4671'),
     ('engler', 'e', 'engler@lcs.mit.edu'),
     ('engler', 'e', 'engler@stanford.edu'),
     ('eroberts', 'e', 'eroberts@cs.stanford.edu'),
     ('eroberts', 'p', '650-723-3642'),
     ('eroberts', 'p', '650-723-6092'),
     ('fedkiw', 'e', 'fedkiw@cs.stanford.edu'),
     ('hager', 'e', 'hager@cs.jhu.edu'),
     

In [314]:
#email_pat = re.compile(r'\b(\w+\.?\w+?)\s*(@|\bat\b)\s*(\w+\.?\w+?)(.edu|.com)\b',re.I)
email_pat = re.compile(r'\b(\w+\.?\w+?)\s*(@|\bat\b|\(at\)|WHERE|&#x40;)\s*(\w+)(\.|\;|\sdo?t\s|\sDOM\s)?\s*(\w+)?(\.|\;|\sdo?t\s|\sDOM\s)?\s*(edu|com)\b',re.I)
email_pat_1 = re.compile(r'obfuscate\(\'(\w+\.edu)\',\'(\w+)\'\)')
email_pat_2 = re.compile(r'\b(\w+\.?\w+?)\s*\(followed by\s*(&ldquo;)?\"?(@)\s*(\w+)(\.)?\s*(\w+)?(\.)?\s*(edu|com)\b',re.I)
email_pat_3 = re.compile(r'([A-Za-z]-){1,}@-([A-Za-z]-){1,}\.-([A-Za-z]-?){1,}')
#email_pat = re.compile(r'(\w+)')
line1 = 'Email: ashishg @ stanford.edu'
line2='cheriton at cs.stanford.edu'
line3='dabo@cs.stanford.edu'
line4='jurafsky(at)cs.stanford.edu'
line5='jurafsky at csli dot stanford dot edu'
line19='Email: subh at stanford dot edu'
line6='obfuscate(\'stanford.edu\',\'jurafsky\')</script>'
line7='eroberts@cs.stanford.edu'
line8='nick.parlante@cs.stanford.edu'
line9='uma at cs.Stanford.EDU'
line10='jks at robotics;stanford;edu'#????
line11='<dd>	<em>ada&#x40;graphics.stanford.edu</em>'
line12='<dd>	<em>melissa&#x40;graphics.stanford.edu</em>'
line13='<address>engler WHERE stanford DOM edu</address>'
line14='hager at cs dot jhu dot edu'
line15=' <td class="value">ouster (followed by &ldquo;@cs.stanford.edu&rdquo;)</td>'
line16='<td class="value">teresa.lynn (followed by "@stanford.edu")</td>'
line17='email: pal at cs stanford edu'
line18='email: serafim at cs dot stanford dot edu'
line20='email to support at gradiance dt com'
line21='<address>engler WHERE stanford DOM edu</address>'
line22='d-l-w-h-@-s-t-a-n-f-o-r-d-.-e-d-u'
#matches = re.findall(email_pat,line)
matches = email_pat.findall(line1)
matches_1 = email_pat_1.findall(line6)
matches_3 = email_pat_3.findall(line22)
print(matches_3)

for m in matches:
    #[('ashishg', '@', 'stanford',vV  '', '', '.', 'edu')]
    #first we need to remove '' from m
    #m = [i for i in m if(len(str(i))!=0)]
    #join the half 
    #half_email = '.'.join(m[2::2])
    half_email = '.'.join(i for i in m[2::2]if(len(str(i)))!=0)
    print half_email  
    email = '%s@%s' % (m[0],half_email.lower())
    print(email)
   

[('h-', 'd-', 'u')]
stanford.edu
ashishg@stanford.edu


In [249]:
phone1='TEL +1-650-723-0293'
phone2='Phone:(650) 723-0293'
phone3='Tel (+1): 650-723-0293'
phone4='<a href="contact.html">TEL</a> +1&thinsp;650&thinsp;723&thinsp;0293'
answer='650-723-0293'
phone5='1342176593'
phone_pat = re.compile(r'\b(\d{3})(\-|\s+|&thinsp;|\)\s*)(\d{3})(\-|\s+|&thinsp;)(\d{4})\b')
phone_matches = phone_pat.findall(phone5)
for m in phone_matches:
    print m
    phone = '-'.join(m[::2])
    print phone

In [235]:
m[::2]

('650', '723', '0293')